In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import r2_score
import pickle

df= pd.read_csv('Datos/Limpios/df_valoracion.csv')
#elimino la columna "Precio/Venta"
df.drop(columns=['Precio/Venta'], inplace=True) 
df.drop(columns=['PER'], inplace=True) 

In [22]:
lista_variables= ['growth_stage', 'startup', 'b2b_b2c', 'Nombre_sabi']
df = df.drop(lista_variables, axis=1)
X_train, X_test, y_train, y_test = train_test_split(df.drop('valuation_2022', axis=1), 
                                                    df['valuation_2022'], 
                                                    test_size=0.2, 
                                                    random_state=0)

In [23]:
print(X_train.shape, X_test.shape)
print(X_train.columns)

(48, 19) (12, 19)
Index(['Total activo mil EUR_2021', 'Activo circulante mil EUR_2021',
       'Fondos propios mil EUR_2021',
       'Total pasivo y capital propio mil EUR_2021',
       'Fondo de maniobra mil EUR_2021', 'Deudores mil EUR_2021',
       'Beneficio neto mil EUR', 'Ingresos de explotacion mil EUR_2021',
       'Importe neto Cifra de Ventas mil EUR_2021',
       'Pasivo liquido mil EUR_2021', 'Total pasivo_2021', 'total_funding',
       'Pasivo fijo mil EUR_2021', 'Gastos financieros mil EUR_2021',
       'Resultado Explotacion mil EUR_2021', 'EBIT mil EUR_2021',
       'Resultado del Ejercicio mil EUR_2021',
       'Result. ordinarios antes Impuestos mil EUR_2021',
       'Importe neto Cifra de Ventas mil EUR_ratio'],
      dtype='object')


--------------------------------

In [37]:
y_train_log = np.log(y_train)


models = {'Linear Regression': LinearRegression(), 
          'Decision Tree': DecisionTreeRegressor(random_state=42), 
          'Random Forest': RandomForestRegressor(random_state=42)}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='r2')
    print(f"{name} R2 score: {scores.mean():.2f} (+/- {scores.std():.2f})")
    print(f"{name} MAE score: {-cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').mean():.2f} (+/- {-cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').std():.2f})")


results = []

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='r2')
    mae_scores = -cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error')
    result = {'model': name,
              'R2 score': scores.mean().round(2),
              'R2 max': scores.max().round(2),
              'MAE score': -cross_val_score(model, X_train, y_train_log.ravel(), cv=5, scoring='neg_mean_absolute_error').mean().round(2)
              }
    results.append(result)    

Linear Regression R2 score: -87.07 (+/- 102.32)
Linear Regression MAE score: 5.40 (+/- -4.08)
Decision Tree R2 score: -1.02 (+/- 1.19)
Decision Tree MAE score: 1.82 (+/- -0.34)
Random Forest R2 score: 0.06 (+/- 0.42)
Random Forest MAE score: 1.23 (+/- -0.18)


In [38]:
results

[{'model': 'Linear Regression',
  'R2 score': -87.07,
  'R2 max': -0.14,
  'MAE score': 5.4},
 {'model': 'Decision Tree',
  'R2 score': -1.02,
  'R2 max': 0.1,
  'MAE score': 1.82},
 {'model': 'Random Forest',
  'R2 score': 0.06,
  'R2 max': 0.44,
  'MAE score': 1.23}]

In [26]:
linear_regression_mae = results[0]['MAE score']
linear_regression_r2 = results[0]['R2 score']
decision_tree_mae = results[1]['MAE score']
decision_tree_r2 = results[1]['R2 score']
random_forest_mae = results[2]['MAE score']
random_forest_r2 = results[2]['R2 score']

In [27]:
# stacking 
level0 = [
    ('knn', KNeighborsRegressor()),
    ('cart', DecisionTreeRegressor()),
    ('svm', SVR()),
    ('lr', LinearRegression())
]

# Define the level 1 model with a transformed target regressor
level1 = TweedieRegressor(max_iter=10000, alpha=0.5, link='log', power=0)

# Define the stacked model
stacked_model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

# Fit the model using cross-validation
scores = -cross_val_score(stacked_model, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(stacked_model, X_train, y_train_log, cv=5, scoring='r2')
avg_mae2 = scores.mean()
avg_r22 = scores2.mean()

# Fit the model on all the training data
stacked_model.fit(X_train, y_train_log)

# Make predictions on the test data and transform the predictions to the original scale
y_pred = np.exp(stacked_model.predict(X_test))

# Evaluate the model on the test data
mae2 = mean_absolute_error(y_test, y_pred)
r22 = r2_score(y_test, y_pred)

# Print the results
print(f"Cross-validation MAE: {avg_mae2:.2f}")
print(f"Cross-validation R^2: {avg_r22:.2f}")
print(f"MAE: {mae2:.2f}")
print(f"R^2: {r22:.2f}")

Cross-validation MAE: 1.45
Cross-validation R^2: -0.19
MAE: 4.27
R^2: 0.69


In [28]:
#Bagging Regressor
br = BaggingRegressor(estimator=RandomForestRegressor(), n_estimators=100, random_state=0)

scores = cross_val_score(br, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(br, X_train, y_train_log, cv=5, scoring='r2')
avg_mae7 = -scores.mean()
avg_r27 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
br.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(br.predict(X_test))

# evaluar el modelo en los datos de prueba
mae7 = mean_absolute_error(y_test, y_pred)
r27 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae7}")
print(f'R2: {avg_r27}')
print(f"MAE: {mae7}")
print(f"R^2: {r27}")

Cross-validation MAE: 1.1231520336461311
R2: 0.2430102107934462
MAE: 5.7132204295869515
R^2: 0.45778841923516644


In [39]:
# adaboost
ab = AdaBoostRegressor(random_state=0)
scores = cross_val_score(ab, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(ab, X_train, y_train_log, cv=5, scoring='r2')
avg_mae9 = -scores.mean()
avg_r29 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
ab.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(ab.predict(X_test))

# evaluar el modelo en los datos de prueba
mae9 = mean_absolute_error(y_test, y_pred)
r29 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae9}")
print(f'R2: {avg_r29}')
print(f"MAE: {mae9}")
print(f"R^2: {r29}")

Cross-validation MAE: 1.2309672046566358
R2: -0.07160222117883137
MAE: 6.228607340858656
R^2: 0.2157599341201043


In [30]:
# gradient boosting
gb = GradientBoostingRegressor(random_state=0)
scores = cross_val_score(gb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2= cross_val_score(gb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae10 = -scores.mean()
avg_r210 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
gb.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(gb.predict(X_test))

# evaluar el modelo en los datos de prueba
mae10 = mean_absolute_error(y_test, y_pred)
r210 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae10}")
print(f'R2: {avg_r210}')
print(f"MAE: {mae10}")
print(f"R^2: {r210}")

Cross-validation MAE: 1.2567382482593827
R2: -0.14566589832116833
MAE: 6.014310342909745
R^2: 0.5075392058811826


In [31]:
# xgboost
xgb = XGBRegressor(random_state=0)
scores = cross_val_score(xgb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(xgb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae11 = -scores.mean()
avg_r211 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
xgb.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(xgb.predict(X_test))

# evaluar el modelo en los datos de prueba
mae11 = mean_absolute_error(y_test, y_pred)
r211 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae11}")
print(f'R2: {avg_r211}')
print(f"MAE: {mae11}")
print(f"R^2: {r211}")

Cross-validation MAE: 1.5130119408615144
R2: -0.5059833935267388
MAE: 9.019712389906248
R^2: -0.4242321267241538


In [32]:
# catboost
cb = CatBoostRegressor(random_state=0)
scores = cross_val_score(cb, X_train, y_train_log, cv=5, scoring='neg_mean_absolute_error')
scores2 = cross_val_score(cb, X_train, y_train_log, cv=5, scoring='r2')
avg_mae12 = -scores.mean()
avg_r212 = scores2.mean()

# ajustar el modelo a todos los datos de entrenamiento
cb.fit(X_train, y_train_log)
# hacer predicciones en los datos de prueba y desescalar las predicciones
y_pred = np.exp(cb.predict(X_test))

# evaluar el modelo en los datos de prueba
mae12 = mean_absolute_error(y_test, y_pred)
r212 = r2_score(y_test, y_pred)

# imprimir los resultados
print(f"Cross-validation MAE: {avg_mae12}")
print(f'R2: {avg_r212}')
print(f"MAE: {mae12}")
print(f"R^2: {r212}")

Learning rate set to 0.024422
0:	learn: 1.7083828	total: 13.9ms	remaining: 13.8s
1:	learn: 1.6951694	total: 14.5ms	remaining: 7.23s
2:	learn: 1.6804947	total: 15.2ms	remaining: 5.06s
3:	learn: 1.6666201	total: 15.9ms	remaining: 3.95s
4:	learn: 1.6535104	total: 16.6ms	remaining: 3.3s
5:	learn: 1.6403839	total: 17.2ms	remaining: 2.85s
6:	learn: 1.6258364	total: 17.7ms	remaining: 2.52s
7:	learn: 1.6122665	total: 18.3ms	remaining: 2.27s
8:	learn: 1.6015587	total: 18.8ms	remaining: 2.07s
9:	learn: 1.5903318	total: 19.4ms	remaining: 1.92s
10:	learn: 1.5803477	total: 20ms	remaining: 1.79s
11:	learn: 1.5665873	total: 20.5ms	remaining: 1.69s
12:	learn: 1.5548487	total: 21ms	remaining: 1.6s
13:	learn: 1.5458566	total: 21.5ms	remaining: 1.52s
14:	learn: 1.5336236	total: 22.1ms	remaining: 1.45s
15:	learn: 1.5221636	total: 22.7ms	remaining: 1.39s
16:	learn: 1.5105787	total: 23.2ms	remaining: 1.34s
17:	learn: 1.4996690	total: 23.7ms	remaining: 1.29s
18:	learn: 1.4891239	total: 24.2ms	remaining: 1.25

In [33]:
import pickle

In [34]:
# se crean modelos con todos los datos
x= df.drop(columns=['valuation_2022'])
y= df['valuation_2022']

br = BaggingRegressor(estimator=RandomForestRegressor(), n_estimators=100, random_state=0)
br.fit(x,y)

ab = AdaBoostRegressor(random_state=0)
ab.fit(x,y)

rf=RandomForestRegressor(random_state=42)
rf.fit(x,y)

xgb = XGBRegressor(random_state=0)
xgb.fit(x,y)

cb = CatBoostRegressor(random_state=0)
cb.fit(x,y)

Learning rate set to 0.02625
0:	learn: 30.8776615	total: 872us	remaining: 872ms
1:	learn: 30.6886334	total: 1.52ms	remaining: 760ms
2:	learn: 30.5114075	total: 2.15ms	remaining: 715ms
3:	learn: 30.2462918	total: 2.81ms	remaining: 699ms
4:	learn: 30.0723852	total: 3.28ms	remaining: 653ms
5:	learn: 29.8157415	total: 3.88ms	remaining: 642ms
6:	learn: 29.6336050	total: 4.49ms	remaining: 637ms
7:	learn: 29.5472681	total: 4.75ms	remaining: 589ms
8:	learn: 29.3264856	total: 5.37ms	remaining: 592ms
9:	learn: 29.1229406	total: 5.96ms	remaining: 591ms
10:	learn: 28.8694568	total: 6.55ms	remaining: 589ms
11:	learn: 28.7151470	total: 7.23ms	remaining: 595ms
12:	learn: 28.5094699	total: 8.1ms	remaining: 615ms
13:	learn: 28.3483051	total: 9.08ms	remaining: 639ms
14:	learn: 28.1559275	total: 9.74ms	remaining: 640ms
15:	learn: 28.0032485	total: 10.4ms	remaining: 638ms
16:	learn: 27.8057723	total: 11ms	remaining: 638ms
17:	learn: 27.6551581	total: 11.7ms	remaining: 640ms
18:	learn: 27.4836233	total: 12

In [35]:
# se guardan los modelos que mejor resultado han dado
# se crea carpeta de modelos si no existe
if not os.path.exists('modelos'):
    os.makedirs('modelos')

# dentro de esa carpeta se crea otra carpeta para los modelos de regresion
if not os.path.exists('modelos/regresion'):
    os.makedirs('modelos/regresion')

# se guarda el modelo de bagging  en la carpeta de modelos
pickle.dump(br, open('modelos/regresion/br_model.pkl', 'wb'))

# se guarda el modelo de adaboost  en la carpeta de modelos
pickle.dump(ab, open('modelos/regresion/ad_model.pkl', 'wb'))

# se guarda el modelo de gradientboost  en la carpeta de modelos
pickle.dump(rf, open('modelos/regresion/rf_model.pkl', 'wb'))

# se guarda el modelo de xgboost  en la carpeta de modelos
pickle.dump(xgb, open('modelos/regresion/xgb_model.pkl', 'wb'))

# se guarda el modelo de catboost  en la carpeta de modelos
pickle.dump(cb, open('modelos/regresion/cb_model.pkl', 'wb'))

**TABLA DE RESULTADOS**

In [36]:
models = pd.DataFrame([
    {'Model': 'Linear Regression', 'MAE': linear_regression_mae, 'R2 score': linear_regression_r2},
    {'Model': 'Decision Tree Regressor', 'MAE': decision_tree_mae, 'R2 score': decision_tree_r2},
    {'Model': 'Random Forest Regressor', 'MAE': random_forest_mae, 'R2 score': random_forest_r2},
    {'Model': 'Stacking Regressor', 'MAE': avg_mae2, 'R2 score': avg_r22},
    {'Model': 'Bagging Regressor', 'MAE': avg_mae7, 'R2 score': avg_r27},
    {'Model': 'AdaBoost Regressor', 'MAE': avg_mae9, 'R2 score': avg_r29},
    {'Model': 'GradientBoost Regressor', 'MAE': avg_mae10, 'R2 score': avg_r210},
    {'Model': 'XGBoost', 'MAE': avg_mae11, 'R2 score': avg_r211},
    {'Model': 'CatBoost', 'MAE': avg_mae12, 'R2 score': avg_r212}
    # {'Model': 'AutoML', 'MAE': avg_mae17, 'Score': avg_r217}
])

models = models.sort_values(by='R2 score', ascending=False)
models = models.reindex(columns=['Model', 'R2 score', 'MAE'])
models

,Model,R2 score,MAE
4,Bagging Regressor,0.243010,1.123152
2,Random Forest Regressor,0.060000,1.230000
8,CatBoost,0.022177,1.217499
5,AdaBoost Regressor,-0.071602,1.230967
6,GradientBoost Regressor,-0.145666,1.256738
3,Stacking Regressor,-0.187699,1.447696
7,XGBoost,-0.505983,1.513012
1,Decision Tree Regressor,-1.020000,1.820000
0,Linear Regression,-87.070000,5.400000
